# Задание 1 (5 балла)

Имплементируйте алгоритм Леска (описание есть в семинаре) и оцените качество его работы на датасете `data/corpus_wsd_50k.txt`

В качестве метрики близости вы должны попробовать два подхода:

1) Jaccard score на множествах слов (определений и контекста)
2) Cosine distance на эмбедингах sentence_transformers

В качестве метрики используйте accuracy (% правильных ответов). Предсказывайте только многозначные слова в датасете

Контекст вы можете определить самостоятельно (окно вокруг целевого слова или все предложение). Также можете поэкспериментировать с предобработкой для обоих методов.

In [ ]:
!mkdir data
!wget https://github.com/mannefedov/compling_nlp_hse_course/raw/master/data/corpus_wsd_50k.txt.zip -P data
!unzip -o data/corpus_wsd_50k.txt.zip -d data/

In [ ]:
!pip install nltk

In [ ]:
import nltk
nltk.download('wordnet')
from nltk.corpus import wordnet as wn
nltk.download('punkt_tab')
from nltk import sent_tokenize
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
nltk.download("stopwords")
english_stopwords = stopwords.words("english")
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()

import re

In [ ]:
corpus_wsd = []
corpus = open('data/corpus_wsd_50k.txt').read().split('\n\n')
for sent in corpus:
    corpus_wsd.append([s.split('\t') for s in sent.split('\n')])
#корпус содержит спискок предложений со списками слов
#формат слова: список с элементами имя значения, лемма, словоформа из текста

In [ ]:
def tokenizer(corpus_wsd):
    tokenized_sentences = []
    words_in_sentences = []
    for sentence in corpus_wsd[:-1]: #последнее предложение - список с пустым списком, удаляю его
        new_sentence = []
        new_words = []
        for el in sentence:
            if not re.match(r'\W+', el[1]) and not el[1] in english_stopwords:
                new_sentence.append(el[1])
                new_words.append(el[2].lower())
        if new_sentence:
            tokenized_sentences.append(new_sentence)
            words_in_sentences.append(new_words)
    return tokenized_sentences, words_in_sentences

In [ ]:
def preprocessing(sentence, words):
    package = []
    for i in range(len(sentence)):
        lemma = sentence[i]
        token = words[i]
        context = sentence[max(0, i-7): i+8]
        context.remove(lemma)
        synsets = wn.synsets(token)
        syn_names = []
        syn_defins = []
        if len(synsets) >= 1:
            for synset in synsets:
                syn_names.append(synset.name())
                syn_defins.append([lemmatizer.lemmatize(word_from_def.strip(',.:;()*"-'))
                for word_from_def in synset.definition().split() if word_from_def.strip(',.:;()*"-')
                not in english_stopwords and word_from_def.strip(',.:;()*"-')])
            package.append([token, context, syn_defins, syn_names, lemma])
    return package

In [ ]:
#промежуточная проверка
tokenized_sentences, words_in_sentences = tokenizer(corpus_wsd)
package = preprocessing(tokenized_sentences[40000], words_in_sentences[40000])

print(tokenized_sentences[40000])
print(*package, sep='\n')

In [ ]:
def jaccard(package: list) -> list:
    best_meaning = []
    for word_inf in package:
        token, context, syn_defins, syn_names, lemma = word_inf

        context_set = set(context)
        jac_result = 0.
        best_name = ''
        for i in range(len(syn_defins)):
            defin_set = set(syn_defins[i])
            intersection = context_set & defin_set
            union = context_set | defin_set
            if union:
                jac = len(intersection) / len(union)
            else:
                jac = 0
            if jac == 0:
                continue
            elif jac > jac_result:     #ЧТО ЛУЧШЕ > или >= ??? Лучше >
                jac_result = jac
                best_name = syn_names[i]
        if best_name == '':
            try:
                best_name = syn_names[0]
            except:
                pass



        best_meaning.append([lemma, best_name])

    return best_meaning

In [ ]:
#промежуточная проверка
tokenized_sentences, words_in_sentences = tokenizer(corpus_wsd)
all_best_meanings_jac = []


for i in range(len(tokenized_sentences)):
    package = preprocessing(tokenized_sentences[i], words_in_sentences[i])
    best_meaning = jaccard(package)
    all_best_meanings_jac.append(best_meaning)
print(all_best_meanings_jac)

In [ ]:
corpus_red = []
for sent in corpus_wsd[:-1]:
    sent_new = []
    for word in sent:
        if not re.match(r'\W+', word[1]) and not word[1] in english_stopwords:
            try:
                name = wn.lemma_from_key(word[0]).synset().name()
            except:
                name = word[0]

            word_new = [word[1], name]
            sent_new.append(word_new)
    if sent_new:
        corpus_red.append(sent_new)

In [ ]:
!pip install scikit-learn
!pip install numpy

In [ ]:
from sklearn.metrics.pairwise import cosine_distances, cosine_similarity
import numpy as np

In [ ]:
!python -m pip install torch torchvision torchaudio
!python -m pip install sentence_transformers transformers accelerate -U

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer

In [ ]:
model = SentenceTransformer('sentence-transformers/all-mpnet-base-v2')
embed = model.encode

In [ ]:
#если я не ошибаюсь, отсутствие лемматизации слов в дефиниции не должно сильно повлиять на векторы
#но это может сильно увеличить время, поэтому я уберу лемматизацию
def preprocessing_for_cos(sentence, words):
    package = []
    for i in range(len(sentence)):
        lemma = sentence[i]
        token = words[i]
        context = sentence[max(0, i-7): i+8]
        context.remove(lemma)
        synsets = wn.synsets(token)
        syn_names = []
        syn_defins = []
        if len(synsets) >= 1:
            for synset in synsets:
                syn_names.append(synset.name())
                syn_defins.append([word_from_def.strip(',.:;()*"-')
                for word_from_def in synset.definition().split() if word_from_def.strip(',.:;()*"-')
                not in english_stopwords and word_from_def.strip(',.:;()*"-')])
            package.append([token, context, syn_defins, syn_names, lemma])
    return package

In [ ]:
'''ЭТО РАБОТАЛО ОЧЕНЬ МЕДЛЕННО
def cossine_dist(package: list) -> list:
    best_meaning = []
    for word_inf in package:
        token, context, syn_defins, syn_names, lemma = word_inf
        context_emb = embed(' '.join(context))

        cos_result = -1.0
        best_name = ''
        for i in range(len(syn_defins)):
            defin_emb = embed(' '.join(syn_defins[i]))
            cos = cosine_similarity(context_emb.reshape(1, -1), defin_emb.reshape(1, -1))[0][0]
            if cos == -1.0:
                continue
            elif cos > cos_result:
                cos_result = cos
                best_name = syn_names[i]
            if best_name == '':
                try:
                    best_name = syn_names[0]
                except:
                    pass

        best_meaning.append([lemma, best_name])

    return best_meaning

    ЭТО РАБОТАЛО ОЧЕНЬ МЕДЛЕННО'''

def cossine_dist_batch(package: list) -> list:
    best_meaning = []
    for word_inf in package:
        token, context, syn_defins, syn_names, lemma = word_inf

        #список текстов: сначала общий контекст, потом все дефиниции
        texts = [' '.join(context)] + [' '.join(defin) for defin in syn_defins]

        # эмбедим одним батчем
        embs = embed(texts)
        context_emb = embs[0].reshape(1, -1)
        defin_embs = embs[1:]

        sims = cosine_similarity(context_emb, defin_embs)[0]

        best_idx = sims.argmax()
        best_name = syn_names[best_idx]

        best_meaning.append([lemma, best_name])

    return best_meaning


In [ ]:
#промежуточная проверка
package_test = preprocessing(tokenized_sentences[40000], words_in_sentences[40000])


best_meaning_j = jaccard(package_test)
best_meaning_c = cossine_dist_batch(package_test)

print(best_meaning_j)
print(best_meaning_c)
print(corpus_red[40000])

In [ ]:
from tqdm import tqdm
#!!!!!!!ПО ВОЗМОЖНОСТИ НЕ ЗАПУСКАТЬ СЛЕДУЮЩУЮ ЯЧЕЙКУ, ОЧЕНЬ ДОЛГО (около 2,5 часов)!!!!!!!!!

In [ ]:
all_best_meanings_cos = []
for i in tqdm(range(len(tokenized_sentences)), desc="Processing sentences"):
    package = preprocessing_for_cos(tokenized_sentences[i], words_in_sentences[i])
    best_meaning = cossine_dist_batch(package)
    all_best_meanings_cos.append(best_meaning)

print(all_best_meanings_cos)

In [ ]:
#еще одна проверочка
print(len(corpus_red))
print(len(all_best_meanings_jac))
print(len(all_best_meanings_cos))

In [ ]:
#я не знаю, в чем проблема, но лучше точность показывает на 20.000, чем на 10.000, 30.000, 40.000...

corp_len = 0
cos_count = 0
jac_count = 0
for i in range(20000):
    cur_corp_sent = corpus_red[i]
    corp_len += len(cur_corp_sent)
    cur_jac_sent = all_best_meanings_jac[i]
    cur_cos_sent = all_best_meanings_cos[i]


    for targ in cur_corp_sent:
        for jac in cur_jac_sent:
            if targ[0] in jac[0] or jac[0] in targ[0]:
                if targ[1] == jac[1]:
                    jac_count += 1
        for cos in cur_cos_sent:
            if targ[0] in cos[0] or cos[0] in targ[0]:
                if targ[1] == cos[1]:
                    cos_count += 1
jac_accuracy = jac_count / corp_len
cos_accuracy = cos_count / corp_len


In [ ]:
print(f'Jaccard Accuracy: {jac_accuracy * 100}%')
print(f'Cossine distances Accuracy: {cos_accuracy * 100}%')

# Задание 2 (5 балла)
Попробуйте разные алгоритмы кластеризации на датасете - `https://github.com/nlpub/russe-wsi-kit/blob/initial/data/main/wiki-wiki/train.csv`

Используйте код из семинара как основу. Используйте ARI как метрику качества.

Попробуйте все 4 алгоритма кластеризации, про которые говорилось на семинаре. Для каждого из алгоритмов попробуйте настраивать гиперпараметры (посмотрите их в документации). Прогоните как минимум 5 экспериментов (не обязательно успешных) с разными параметрами на каждый алгоритме кластеризации и оцените: качество кластеризации, скорость работы, интуитивность параметров.

Помимо этого также выберите 1 дополнительный алгоритм кластеризации отсюда - https://scikit-learn.org/stable/modules/clustering.html , опишите своими словами принцип его работы  и проделайте аналогичные эксперименты.

In [ ]:
!pip install pandas

In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv('https://raw.githubusercontent.com/nlpub/russe-wsi-kit/initial/data/main/wiki-wiki/train.csv', sep='\t')

In [ ]:
grouped_df = df.groupby('word')[['word', 'context', 'gold_sense_id']]

In [ ]:
from sklearn.cluster import KMeans, DBSCAN, AffinityPropagation, AgglomerativeClustering, BisectingKMeans
import numpy as np
from sklearn.metrics import adjusted_rand_score

In [ ]:
import time

In [ ]:
class ClusterBoss:

    def __init__(self, grouped_df, embed, param_list: list[dict], name: str = 'KM'):
        self.__grouped_df = grouped_df
        self.__param_list = param_list
        self.__name = name
        self.__embed = embed

        # для кеширования
        self.__X_by_key = {}
        self.__gold_by_key = {}
        self.__group_keys = list(self.__grouped_df.groups.keys())

        print(f'Кластер типа {name} создан')

        self.__prepare_embeddings()

    def __prepare_embeddings(self):
        for key in tqdm(self.__group_keys, desc="Кеширование эмбеддингов", leave=True):
            texts = self.__grouped_df.get_group(key)['context'].values

            X = np.zeros((len(texts), 768))
            for i, text in enumerate(texts):
                X[i] = self.__embed(text)

            self.__X_by_key[key] = X
            self.__gold_by_key[key] = self.__grouped_df.get_group(key)['gold_sense_id'].values

    def cluster_maker(self):
        clust_d = {
            'KM': KMeans,
            'BKM': BisectingKMeans,
            'DB': DBSCAN,
            'AC': AgglomerativeClustering,
            'AP': AffinityPropagation
        }

        cluster_type = clust_d[self.__name]
        clusters = []

        for params in self.__param_list:
            cluster = cluster_type(**params)
            clusters.append(cluster)

        return clusters

    def cluster_tester(self):

        start_time = time.time()
        results = []

        for cluster_model in self.cluster_maker():
            ARI_scores = []


            for key in tqdm(self.__group_keys, desc=f"{self.__name} — {cluster_model}", leave=False):
                X = self.__X_by_key[key]
                gold_labels = self.__gold_by_key[key]

                cluster_model.fit(X)
                labels = np.array(cluster_model.labels_)
                score = adjusted_rand_score(gold_labels, labels)
                ARI_scores.append(score)

            results.append(np.mean(ARI_scores))  # средний ARI по всем словам
        elapsed_time = time.time() - start_time  # конец замера
        print(f"\nВремя работы алгоритма кластеризации: {elapsed_time:.2f} сек")

        return results

    def result_printer(self):
        results = self.cluster_tester()
        print(f"\nРезультаты для {self.__name}:")
        for i, score in enumerate(results):
            print(f"Параметры #{i+1}: ARI = {score}")


In [ ]:
param_list_km = [
    {'n_clusters': 2, 'init': 'k-means++'},
    {'n_clusters': 3, 'init': 'random', 'random_state': 42},
    {'n_clusters': 4, 'init': 'random'},
    {'n_clusters': 4, 'init': 'k-means++'},
    {'n_clusters': 5, 'init': 'random'},
]

universal_claster_tester = ClusterBoss(grouped_df, embed, param_list_km, name='KM')
universal_claster_tester.result_printer()

KMeans

- Качество кластеризации:

Параметры #1: ARI = 0.03127852963003641
{'n_clusters': 2, 'init': 'k-means++'}

Параметры #2: ARI = 0.05480462405621067
{'n_clusters': 3, 'init': 'random', 'random_state': 42},

Параметры #3: ARI = 0.061853664124391895
{'n_clusters': 4, 'init': 'random'}

Параметры #4: ARI = 0.04471676867015897
{'n_clusters': 4, 'init': 'k-means++'},

Параметры #5: ARI = 0.046457517131157644
{'n_clusters': 5, 'init': 'random'}

- Скорость работы: 0.48 сек


- Интуитивность параметров:

я выбрала настраивать параметры n_clusters, random_state и init, но не очень поняла из описания, нужно ли вообще выбирать n_clusters при random.
1) Он рандомно выбирает именно точки возникновения кластеров, а количество определяет n_clusters (что было бы лочично, учитывая то, что он противопоставляется k-means++, который выбирает оптимальные точки образования кластеров)?
2) Или он выбирает количество кластеров, а рандомные точки можно выбрать, используя другой параметр: random_state?

   Для меня не очень интутивно понятно

In [ ]:
param_list_bkm = [
    {'n_clusters': 2, 'init': 'k-means++', 'bisecting_strategy' : 'biggest_inertia'},
    {'n_clusters': 3, 'init': 'random', 'random_state': 42, 'bisecting_strategy' : 'biggest_inertia'},
    {'n_clusters': 4, 'init': 'random'},
    {'n_clusters': 4, 'init': 'k-means++'},
    {'n_clusters': 5, 'init': 'random'},
]

universal_claster_tester = ClusterBoss(grouped_df, embed, param_list_bkm, name='BKM')
universal_claster_tester.result_printer()

BissectingKMeans

- Мой дополнительный алгоритм.
Похож на KMeans, для него указывается количество кластеров, но они создаются не одновременно, а делением, как при иерархической кластеризации. Кластер многократно разделяется на два новых кластера, пока не будет достигнуто целевое количество кластеров.

- Качество кластеризации:
Параметры #1: ARI = 0.0008880278863254865 {'n_clusters': 2, 'init': 'k-means++', 'bisecting_strategy' : 'biggest_inertia'}

Параметры #2: ARI = 0.07487141079003236 {'n_clusters': 3, 'init': 'random', 'random_state': 42, 'bisecting_strategy' : 'biggest_inertia'}

Параметры #3: ARI = 0.016652509430728817 {'n_clusters': 4, 'init': 'random'}

Параметры #4: ARI = 0.07658255215506855 {'n_clusters': 4, 'init': 'k-means++'},

Параметры #5: ARI = 0.04502049878699138 {'n_clusters': 5, 'init': 'random'}

- Скорость работы: 0.37 сек

- Интуитивность параметров:

То же самое, что и у KMeans, но есть еще параметр bisecting_strategy. Интуитивно понятно, что дефолтная стратегия biggest_inertia медленнее, но иногда точнее.

Смотря на тесты двух этих моделей, можно, разве что, сделать вывод, что 2 кластера - это мало, а рандомные параметры иногда дают лучшие результаты.


Еще видно, что вторая модель быстрее.

In [ ]:
param_list_db = [
    {'eps': 0.1,  'min_samples': 2, 'metric': 'cosine'},
    {'eps': 0.3,  'min_samples': 3, 'metric': 'cosine'},
    {'eps': 0.5,  'min_samples': 5, 'metric': 'euclidean'},
    {'eps': 0.7,  'min_samples': 2, 'metric': 'cosine'},
    {'eps': 1.0,  'min_samples': 1, 'metric': 'euclidean'},
]
universal_claster_tester = ClusterBoss(grouped_df, embed, param_list_db, name='DB')
universal_claster_tester.result_printer()

DBSCAN

- Качество кластеризации:
Параметры #1: ARI = 0.03975148102934692 {'eps': 0.1,  'min_samples': 2, 'metric': 'cosine'}

Параметры #2: ARI = -0.001784882962278153 {'eps': 0.3,  'min_samples': 3, 'metric': 'cosine'}

Параметры #3: ARI = -0.011271692824715207 {'eps': 0.5,  'min_samples': 5, 'metric': 'euclidean'}

Параметры #4: ARI = 0.0 {'eps': 0.7,  'min_samples': 2, 'metric': 'cosine'}

Параметры #5: ARI = 0.0 {'eps': 1.0,  'min_samples': 1, 'metric': 'euclidean'}

- Скорость работы: 0.39 сек

- Интуитивность параметров: параметры очень интуитивно понятные, в документации хорошее пояснение (расстояние для поля, количество экземпляров в поле и пр.)

У меня этот алгоритм работает хуже, чем остальные на разных параметрах. До этого я пробовала еще на других параметрах, но там вообще не было ни одного результата > 0, поэтому я переделала вот на этих

In [ ]:
param_list_ap = [
    {'damping': 0.5},
    {'damping': 0.6},
    {'damping': 0.7},
    {'damping': 0.8},
    {'damping': 0.9},
]
universal_claster_tester = ClusterBoss(grouped_df, embed, param_list_ap, name='AP')
universal_claster_tester.result_printer()

AffinityPropagation

- Качество кластеризации:

Параметры #1: ARI = 0.042363774919161074 {'damping': 0.5}

Параметры #2: ARI = 0.042740969848549505 {'damping': 0.6}

Параметры #3: ARI = 0.04154515818974152 {'damping': 0.7}

Параметры #4: ARI = 0.04154515818974152 {'damping': 0.8}

Параметры #5: ARI = 0.05297560306165972 {'damping': 0.9}


- Скорость работы: 0.58 сек

- Интуитивность параметров:

Интуитивно непонятные параметры, как будто бы даже особо нечего настраивать. Хотела измеить affinity на precomputed для интереса, но оказалось, что данные тогда нужно подавать в виде матрицы, для чего мой класс не приспособлен.

In [ ]:
param_list_ac = [
    {'n_clusters': 2, 'metric': 'euclidean', 'linkage' : 'ward'},
    {'n_clusters': 3, 'metric': 'cosine', 'linkage': 'complete'},
    {'n_clusters': 4, 'metric': 'euclidean', 'linkage' : 'ward'},
    {'n_clusters': 4, 'metric': 'manhattan', 'linkage' : 'average'},
    {'n_clusters': 5, 'metric': 'euclidean', 'linkage' : 'single'},
]

universal_claster_tester = ClusterBoss(grouped_df, embed, param_list_ac, name='AC')
universal_claster_tester.result_printer()

AgglomerativeClustering

- Качество кластеризации:

Параметры #1: ARI = -0.011976265536517934 {'n_clusters': 2, 'metric': 'euclidean', 'linkage' : 'ward'}

Параметры #2: ARI = 0.11591119562417704 {'n_clusters': 3, 'metric': 'cosine', 'linkage': 'complete'}

Параметры #3: ARI = 0.038617257915792846 {'n_clusters': 4, 'metric': 'euclidean', 'linkage' : 'ward'}

Параметры #4: ARI = 0.0823422174814261 {'n_clusters': 4, 'metric': 'manhattan', 'linkage' : 'average'}

Параметры #5: ARI = 0.1183189059773965 {'n_clusters': 5, 'metric': 'euclidean', 'linkage' : 'single'}

- Скорость работы: 0.38 сек

- Интуитивность параметров: параметры очень интуитивно понятные, в документации хорошее пояснение, можно настроить и количество кластеров, и способ оценки связности и метрику вычисления.

Последний метод кластеризации тут кажется оптимальным: быстрым, легко настраиваемым, более точным